In [ ]:
""" 为每个group中的结构进行原子对应和晶格匹配 """
from crystmatch import *
import os 

def Enumeration_all_CSM(max_mu,max_strain,tol,max_distance,init,final,csmcar,save_path):
       crystA = load_poscar(filename=init,to_primitive=True)
       crystB = load_poscar(filename=final,to_primitive=True)
       jobname = f"m{max_mu}s{max_strain}"
       voigtA, voigtB, weight_func, ori_rel =  None, None, None, None # load_csmcar(csmcar) 
       check_stoichiometry(crystA[1], crystB[1])   ## 检查元素匹配
       zlcm = np.lcm(len(crystA[1]), len(crystB[1]))  ## 计算最小公倍数，匹配原子数
       if voigtA is None and voigtB is None:
              strain = rmss
              print("Using the root-mean-squared strain (RMSS) to quantify deformation.")
       elif voigtA is not None and voigtB is not None:
              elasticA = voigt_to_tensor(voigtA, cryst=crystA, tol=tol)
              elasticB = voigt_to_tensor(voigtB, cryst=crystB, tol=tol)
              strain = strain_energy_func(elasticA, elasticB)
              print("Using the estimated strain energy (w) to quantify deformation.")
       else:
              raise ValueError("Both voigtA and voigtB must be provided if either is provided.")
       if max_distance is None:
              slmlist, pct_arrs, mulist, strainlist, dlist = enumerate_rep_csm(crystA, crystB, max_mu, max_strain, strain=strain, weight_func=weight_func, tol=tol)
              slm_ind = np.arange(len(slmlist))
       else:
              max_d = max_distance
              jobname += f"d{max_d:.2f}"
              slmlist, slm_ind, pct_arrs, mulist, strainlist, dlist = enumerate_all_csm(crystA, crystB, max_mu, max_strain, max_d, strain=strain, weight_func=weight_func, tol=tol)
       if strain == rmss:
              header = ['csm_id', 'slm_id', 'mu', 'period', 'rmss', 'd']
              data = np.array([np.arange(len(slm_ind)), slm_ind, mulist, zlcm * mulist, strainlist, dlist]).T
       else:
              rmsslist = rmss(deformation_gradient(crystA, crystB, slmlist))[slm_ind]
              header = ['csm_id', 'slm_id', 'mu', 'period', 'w', 'rmss', 'd']
              data = np.array([np.arange(len(slm_ind)), slm_ind, mulist, zlcm * mulist, strainlist, rmsslist, dlist]).T
       table = Table(data, header)
       os.makedirs(save_path)
       save_npz(unique_filename("Saving SLMs and CSMs to", save_path+f"\\CSMLIST-{jobname}.npz"), crystA, crystB, slmlist, slm_ind, pct_arrs, table)
       return data 
max_mu = 2
max_strain = 0.3
tol = 1e-3
max_distance = None
init = r"E:\课题2-成核模拟\NaCl\POSCAR-B1-30GPa.vasp"
final = r"E:\课题2-成核模拟\NaCl\POSCAR-B2-30GPa.vasp"
csmcar = r"E:\课题10-相图\Ga2O3\POSCAR_β.vasp"  ## 是否指定输出
save_path = r"C:\Users\Administrator\Desktop\EXPORT-CSM"
result = Enumeration_all_CSM(max_mu,max_strain,tol,max_distance,init,final,csmcar,save_path)

In [ ]:
import numpy as np 
from ase.io import read, write
from ase import Atoms

def interpolate_structures(poscar1, poscar2, n=20):
    """
    读取两个POSCAR文件，生成n个结构（包含初始和终态）的插值轨迹
    参数:
        poscar1: 初始结构POSCAR文件路径
        poscar2: 终态结构POSCAR文件路径
        n: 插值结构数量（包含两个端点）
    返回:
        一个包含n个ASE Atoms对象的列表
    """
    struct1 = read(poscar1, format="vasp")
    struct2 = read(poscar2, format="vasp")
    assert len(struct1) == len(struct2), "两个POSCAR中的原子数不一致！"
    structures = []
    for alpha in np.linspace(0, 1, n):
        cell = (1 - alpha) * struct1.get_cell() + alpha * struct2.get_cell()
        positions = (1 - alpha) * struct1.get_positions() + alpha * struct2.get_positions()
        new_struct = Atoms(symbols=struct1.get_chemical_symbols(),
                           positions=positions,
                           cell=cell,
                           pbc=True)
        structures.append(new_struct)
    return structures

def normalize_lattice(L1, L2):
    L1 = np.array(L1)
    L2 = np.array(L2)
    assert L1.shape == L2.shape, "矩阵大小必须一致"
    n = L1.shape[0]
    assert np.allclose(L1, np.tril(L1)), "L1 不是下三角矩阵"
    assert np.allclose(L2, np.tril(L2)), "L2 不是下三角矩阵"
    flips = np.ones(n)
    L2_new = L2.copy()
    for j in range(n):
        sign_diff = np.sign(L1[j, j]) != np.sign(L2[j, j])
        if np.any(sign_diff):
            flips[j] = -1
            L2_new[:, j] *= -1
    return L2_new

def lq_decomposition(crystA, crystB):
    Q_A, R_A = np.linalg.qr(crystA[0].T)    
    Q_B, R_B = np.linalg.qr(crystB[0].T)
    R_A_T = R_A.T
    R_B_T = normalize_lattice(R_A.T, R_B.T)
    crystA_new = (R_A_T,) + crystA[1:]
    crystB_new = (R_B_T,) + crystB[1:]
    return crystA_new,crystB_new

def save_poscar_xdatcar(npz_file,save_dir):
    crystA, crystB, slmlist, slm_ind, pct_arrs, table = load_npz(npz_file, verbose=True)
    poscar_mode = 'norot'
    xdatcar_mode = 'norot'
    for i in range(len(slm_ind)):
        slm, p, ks = unzip_csm(i, crystA, crystB, slmlist, slm_ind, pct_arrs)
        ind = i
        if poscar_mode == 'norot' or xdatcar_mode == 'norot':
                    crystA_matched,crystB_matched = csm_to_cryst(crystA, crystB, slm, p, ks, tol=tol,
                                                            orientation='norot', min_t0=True, weight_func=None)
                    import os
                    import shutil
                    if os.path.exists(save_dir+f"\\CSM_{i}"):
                        shutil.rmtree(save_dir+f"\\CSM_{i}")
                    os.makedirs(save_dir+f"\\CSM_{i}")
                    crystA_csm, crystB_csm_norot = lq_decomposition(crystA_matched,crystB_matched)
                    if poscar_mode == 'norot':
                        save_poscar(save_dir+f"\\CSM_{i}\\POSCAR_I", crystA_csm, crystname=f"CSM_{ind:d} initial")
                        save_poscar(save_dir+f"\\CSM_{i}\\POSCAR_F", crystB_csm_norot, crystname=f"CSM_{ind:d} final (no rotation)")
                        traj_structures = interpolate_structures(save_dir+f"\\CSM_{i}\\POSCAR_I", save_dir+f"\\CSM_{i}\\POSCAR_F", n=50)
                        output_xyz = save_dir+f"\\CSM_{i}\\traj.xyz"
                        write(output_xyz, traj_structures,format="extxyz")
                    if xdatcar_mode == 'norot':
                        save_xdatcar(save_dir+f"\\CSM_{i}\\XDATCAR", crystA_matched,crystB_matched, images=50, crystname=f"CSM_{ind:d} (no rotation)")
save_poscar_xdatcar(npz_file="CSMLIST-m2s0.3.npz",save_dir=rf"C:\Users\Administrator\Desktop\EXPORT-CSM")